In [1]:
import pandas as pd
import os
from pathlib import Path

DATA_DIR = Path(os.path.abspath('')).parents[1] / "data"
STOCK_PRICES_DIR_RAW = DATA_DIR / "stocks" / "raw"
TRANSPORT_FARES_DIR = DATA_DIR / "fares"
TRANSPORT_FARES_DIR

PosixPath('/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/fares')

# Transport Fares

In [2]:
bus_and_coach_fares = pd.read_csv(TRANSPORT_FARES_DIR / "raw" / "bus_and_coach_fares.csv", skiprows=7)
rail_fares = pd.read_csv(TRANSPORT_FARES_DIR / "raw" / "rail_fares.csv", skiprows=7)
tfl_fares = pd.read_csv(TRANSPORT_FARES_DIR / "raw" / "TfLHistoricalFares2000to2025.csv")

In [3]:
n_rows = 2024 - int(rail_fares.iloc[0].year) + 1

rail_fares = rail_fares.iloc[:n_rows]
rail_fares["year"] = rail_fares["year"].astype(int)

bus_and_coach_fares = bus_and_coach_fares.iloc[:n_rows]
bus_and_coach_fares["year"] = bus_and_coach_fares["year"].astype(int)

In [4]:
combined_fares = bus_and_coach_fares.merge(rail_fares, on='year')
combined_fares = combined_fares.merge(tfl_fares, on='year', how='outer')
combined_fares.head()

,year,relative_bus_and_coach_fares,relative_rail_fares,singleZ1to4Cash,singleZ1to4OysterPeak,busCash,singleBusOyster,capBusTram,travelcardZ1to4,capZ1to4PAYG,travelcard7DayZ1to4,weeklyBusAndTramPass
0,1987,103.4,100.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1988,110.6,107.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1989,119.3,117.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990,125.9,127.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1991,143.6,141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
combined_fares.to_csv(TRANSPORT_FARES_DIR / "processed" / "combined_transport_fares.csv", index=False)

# Stock Indeces

In [6]:
def preprocess_stock_prices_dataset(fpath: Path) -> None:
    df = pd.read_csv(fpath)
    df["Date"] = pd.to_datetime(df["Date"])
    df["year"] = df.Date.dt.year

    df_processed = df.groupby("year").max().sort_values("year")
    df_processed.columns = [s.strip().lower() for s in df_processed.columns]

    if ("year" not in df_processed.columns):
        df_processed = df_processed.reset_index(drop=False)

    # df_processed.to_csv(fname[:-4] + "_processed.csv", index=False)
    new_fname =  fpath.name[:-4] + "_processed.csv"
    df_processed.to_csv(fpath.parents[1] / "processed" / new_fname, index=False)

    return df_processed

In [7]:
processed_indecis = {}
for fpath in STOCK_PRICES_DIR_RAW.iterdir():
    if ("csv" in str(fpath)):
        print(fpath.parent, fpath.name)
        processed_indecis[fpath.name[:-4]] = preprocess_stock_prices_dataset(fpath)

/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw HangSeng_1985to2025.csv
/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw FTSE100_1985to2025.csv


/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_63016/1530709402.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])
/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_63016/1530709402.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw Nikkei225_1980to2025.csv
/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw SnP500_1980to2025.csv


/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_63016/1530709402.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])
/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_63016/1530709402.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw DAX_1987to2025.csv


/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_63016/1530709402.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


In [8]:
pd.DataFrame(processed_indecis['DAX_1987to2025'].year, columns=['year']).merge(processed_indecis['DAX_1987to2025'][['year', 'high']])

,year,high
0,1987,1000.00
1,1988,1340.41
2,1989,1790.37
3,1990,1968.55
4,1991,1715.80
5,1992,1811.57
6,1993,2266.68
7,1994,2271.11
8,1995,2394.15
9,1996,2909.91


In [9]:
# processed_indecis['DAX_1987to2025'].high
combined_indecis = pd.DataFrame(processed_indecis['DAX_1987to2025'].year, columns=['year'])
for key, df in processed_indecis.items():
    incoming_df = df[['year','high']].rename(columns={'high': key})
    combined_indecis = combined_indecis.merge(incoming_df, on='year', how='outer')

In [10]:
combined_indecis

,year,HangSeng_1985to2025,FTSE100_1985to2025,Nikkei225_1980to2025,SnP500_1980to2025,DAX_1987to2025
0,1980,NaN,NaN,7188.28,141.96,NaN
1,1981,NaN,NaN,8019.14,140.32,NaN
2,1982,NaN,NaN,8026.99,144.65,NaN
3,1983,NaN,NaN,9893.82,173.10,NaN
4,1984,NaN,NaN,11577.44,170.41,NaN
5,1985,1757.29,1413.60,13128.94,213.02,NaN
6,1986,2568.30,1717.60,18936.24,254.85,NaN
7,1987,3949.73,2443.40,26646.43,337.89,1000.00
8,1988,2772.53,1879.30,30050.93,283.95,1340.41
9,1989,3309.64,2426.00,38915.87,360.44,1790.37


In [11]:
combined_indecis.to_csv(STOCK_PRICES_DIR_RAW.parent / "combined_indices.csv", index=False)